In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url = 'https://ucirvinesports.com/sports/mens-basketball/stats/2024-25?path=mbball'

page = requests.get(url) #requests object

soup = BeautifulSoup(page.text, 'html')

In [3]:
table = soup.find_all('table')[1]
print(table)

<table accordion-table="" class="sidearm-table collapse-on-medium accordion" sortable-table="">
<caption class="hide">Overall Individual Statistics</caption>
<thead class="sidearm-primary">
<tr>
<th class="text-left" rowspan="2" scope="col">#</th>
<th class="text-hide" rowspan="2" scope="col">Player</th>
<th rowspan="2" scope="col">GP</th>
<th rowspan="2" scope="col">GS</th>
<th colspan="2" scope="colgroup">Minutes</th>
<th colspan="3" scope="colgroup">FG</th>
<th colspan="3" scope="colgroup">3PT</th>
<th colspan="3" scope="colgroup">FT</th>
<th colspan="2" scope="colgroup">Scoring</th>
<th colspan="4" scope="colgroup">Rebounds</th>
<th rowspan="2" scope="col">PF</th>
<th rowspan="2" scope="col">AST</th>
<th rowspan="2" scope="col">TO</th>
<th rowspan="2" scope="col">STL</th>
<th rowspan="2" scope="col">BLK</th>
<th aria-hidden="true" class="hide-on-large" rowspan="2" scope="col">Bio Link</th>
</tr>
<tr>
<th scope="col">TOT</th>
<th scope="col">AVG</th>
<th scope="col">FGM</th>
<th sco

In [4]:
#extract the table headers
headers = table.find_all('th', attrs={"scope": "col"})
print(headers)
num_headers = len(headers)

#loop through the headers
table_headers = [title.text.strip() for title in headers]

#Move these headers to the back of the list to fix the order
headers_to_move = ["PF", "AST", "AST/G", "TO", "STL", "BLK", "Bio Link"]

fixed_headers = [h for h in table_headers if h not in headers_to_move]  # Keep other headers in original order
moved_headers = [h for h in table_headers if h in headers_to_move]  # Move these to the end

# Combine lists to get the final order
final_headers = fixed_headers + moved_headers
print(final_headers)

[<th class="text-left" rowspan="2" scope="col">#</th>, <th class="text-hide" rowspan="2" scope="col">Player</th>, <th rowspan="2" scope="col">GP</th>, <th rowspan="2" scope="col">GS</th>, <th rowspan="2" scope="col">PF</th>, <th rowspan="2" scope="col">AST</th>, <th rowspan="2" scope="col">TO</th>, <th rowspan="2" scope="col">STL</th>, <th rowspan="2" scope="col">BLK</th>, <th aria-hidden="true" class="hide-on-large" rowspan="2" scope="col">Bio Link</th>, <th scope="col">TOT</th>, <th scope="col">AVG</th>, <th scope="col">FGM</th>, <th scope="col">FGA</th>, <th scope="col">FG%</th>, <th scope="col">3PT</th>, <th scope="col">3PTA</th>, <th scope="col">3PT%</th>, <th scope="col">FTM</th>, <th scope="col">FTA</th>, <th scope="col">FT%</th>, <th scope="col">PTS</th>, <th scope="col">AVG</th>, <th scope="col">OFF</th>, <th scope="col">DEF</th>, <th scope="col">TOT</th>, <th scope="col">AVG</th>]
['#', 'Player', 'GP', 'GS', 'TOT', 'AVG', 'FGM', 'FGA', 'FG%', '3PT', '3PTA', '3PT%', 'FTM', 'FT

In [5]:
#Extract data for columns
column_data = table.find_all('tr')
df = pd.DataFrame(columns = final_headers)

for row in column_data[2:]: 
    row_data = row.find_all('td')
    individual_row_data = [data.get_text(separator = " ").strip() for data in row_data]
    individual_row_data[1] = " ".join(individual_row_data[1].split()).split(" ", 1)[-1]
    if len(individual_row_data) > 1:
        words = individual_row_data[1].split()  # Split into words
    if len(words) > 2:  
        individual_row_data[1] = " ".join(words[2:])  # Keep everything after the first two words
        
    print(individual_row_data)
    
    length = len(df)
    df.loc[length] = individual_row_data



['15', 'Leuchten, Bent', '26', '26', '675', '26.0', '122', '225', '.542', '15', '30', '.500', '151', '183', '.825', '410', '15.8', '65', '165', '230', '8.8', '40', '46', '35', '13', '33', 'View Bio']
['11', 'Tillis, Devin', '25', '25', '724', '29.0', '133', '239', '.556', '30', '77', '.390', '48', '55', '.873', '344', '13.8', '40', '159', '199', '8.0', '45', '49', '39', '15', '6', 'View Bio']
['77', 'Che, Myles', '26', '26', '740', '28.5', '97', '239', '.406', '35', '91', '.385', '91', '102', '.892', '320', '12.3', '10', '55', '65', '2.5', '49', '75', '67', '21', '0', 'View Bio']
['02', 'Hohn, Justin', '26', '26', '766', '29.5', '105', '280', '.375', '47', '124', '.379', '43', '56', '.768', '300', '11.5', '10', '76', '86', '3.3', '53', '97', '65', '41', '1', 'View Bio']
['04', 'Henry, Andre', '9', '9', '226', '25.1', '26', '67', '.388', '5', '25', '.200', '11', '12', '.917', '68', '7.6', '6', '20', '26', '2.9', '14', '10', '11', '12', '0', 'View Bio']
['24', 'Dixon, Jurian', '26', '17'

In [8]:
df

#Export this into a csv
df.to_csv(r'/Users/ryanuyeki/Downloads/asa/uci_player_stats.csv', index=False, encoding = "utf-8")